In [ ]:
from djitellopy import Tello
import cv2
import numpy as np
import time

def detection_center(det):
    """Center x, y normalized to [-0.5, 0.5]"""
    cx = (det[3] + det[5]) / 2.0 - 0.5
    cy = (det[4] + det[6]) / 2.0 - 0.5
    return (cx, cy)

def norm(vec):
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    best_det = None
    min_dist = float('inf')
    for det in detections[0, 0]:
        if det[2] > 0.4 and int(det[1]) == 1:  # 'person'
            center = detection_center(det)
            distance = norm(center)
            if distance < min_dist:
                min_dist = distance
                best_det = det
    return best_det

# Load COCO class names
with open('COCO/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')

COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

model = cv2.dnn.readNet(model='COCO/frozen_inference_graph.pb',
                        config='COCO/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt',
                        framework='TensorFlow')

tello = Tello()
print("Connecting to Tello...")
tello.connect()
print(f"Battery level: {tello.get_battery()}%")
tello.streamon()

print("Press 't' to takeoff, 'q' to land and quit.")
drone_in_air = False

# gain to scale offset to cm
k_x = 80   # cm per unit x-offset (screen left-right)
k_y = 100  # cm per unit y-offset (screen up-down)

# min/max move threshold (cm)
min_move_cm = 20
max_move_cm = 80

def clip(val):
    if abs(val) < min_move_cm:
        return 0
    return max(-max_move_cm, min(max_move_cm, val))

try:
    while True:
        frame = tello.get_frame_read().frame
        frame = cv2.resize(frame, (720, 480))
        h, w, _ = frame.shape

        # Object detection
        blob = cv2.dnn.blobFromImage(frame, size=(300, 300), mean=(104, 117, 123), swapRB=True)
        model.setInput(blob)
        detections = model.forward()
        det = closest_detection(detections)

        if det is not None:
            class_id = int(det[1])
            class_name = class_names[class_id - 1]
            color = COLORS[class_id]

            x1 = int(det[3] * w)
            y1 = int(det[4] * h)
            x2 = int(det[5] * w)
            y2 = int(det[6] * h)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            label = f"{class_name} {det[2]:.2f}"
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

            # Get center offset of person
            cx, cy = detection_center(det)
            move_x = clip(int(k_x * cx))
            move_y = clip(int(k_y * cy))

            if drone_in_air:
                if move_x != 0:
                    if move_x > 0:
                        print(f"Moving right {abs(move_x)} cm")
                        tello.move_right(abs(move_x))
                    else:
                        print(f"Moving left {abs(move_x)} cm")
                        tello.move_left(abs(move_x))

                # if move_y != 0:
                #     if move_y > 0:
                #         print(f"Moving down {abs(move_y)} cm")
                #         tello.move_down(abs(move_y))
                #     else:
                #         print(f"Moving up {abs(move_y)} cm")
                #         tello.move_up(abs(move_y))

                time.sleep(0.3)  # Let the drone stabilize between moves

        # Display the frame
        cv2.imshow("Tello Tracking Feed", frame)

        key = cv2.waitKey(1) & 0xFF

        # Takeoff
        if key == ord('t') and not drone_in_air:
            tello.takeoff()
            drone_in_air = True
            print("Drone is now airborne and hovering.")
            tello.move_up(30)
            print("Drone moved up 30 cm.")

        # Manual fallback (optional)
        if drone_in_air:
            if key == ord('w'):
                print("Manual forward.")
                tello.move_forward(50)
            elif key == ord('a'):
                print("Manual left.")
                tello.move_left(50)
            elif key == ord('s'):
                print("Manual backward.")
                tello.move_back(50)
            elif key == ord('d'):
                print("Manual right.")
                tello.move_right(50)

        # Land and quit
        if key == ord('q'):
            if drone_in_air:
                tello.land()
                print("Drone has landed.")
            break

except Exception as e:
    print(f"Error occurred: {e}")
    if drone_in_air:
        tello.land()

except KeyboardInterrupt:
    print("Keyboard Interrupt. Landing drone.")
    if drone_in_air:
        tello.land()

finally:
    tello.streamoff()
    cv2.destroyAllWindows()

